# Basic Pipeline

Things I want to learn/try:
- Data flow from files (for bigger data/projects)
- Loading and tuning a pre-trained model
- Layer visualisation methods
- Picking out the miss-classified from validation, and displaying them
- Non-sequential models (No excuse yet)

In [ ]:
# The very first time you import keras and seaborn, there's a long delay as
# setup stuff happens
import json
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from keras import layers, models, optimizers
from keras.preprocessing.image import ImageDataGenerator

# This contains a few useful functions for reshaping, plotting etc
import src.helpers as h

In [ ]:
with open("data/shipsnet.json", "r") as f:
    data = json.load(f)

# Data structure
print([key for key in data.keys()])

# Check labels
print(data["labels"][:10], data["labels"][-10:])

# Check labels split
print("True: ", sum([i == 1 for i in data["labels"]]))
print("False: ", sum([i == 0 for i in data["labels"]]))

# Plot an example to check
h.quick_plot_img(data["data"][5])

In [ ]:
# Split the data - in this case just train and test
train_X, train_y, val_X, val_y, test_X, test_y = h.train_test_validation_split(
    data["data"], data["labels"], validation=True
)

# Reformat the split features into (N, 80, 80, 3) shape array
train_X = h.format_imgs(train_X)
val_X = h.format_imgs(val_X)
test_X = h.format_imgs(test_X)

## 2. Make data generators (implements data augmentation steps)

In [ ]:
# Configure data generators
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode="nearest",
)

# For test, obviously no augmentation
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Feed the generators the source data
# DEPRECATION since book: class_mode now auto-detected? Unsure.
# NOTE: Shuffling and creation of a validation set can be handled by the
# generator, by passing the flow method 'shuffle=True' and by passing model.fit
# 'subset = "training"' or 'subset="validation"' respectively
train_generator = train_datagen.flow(
    train_X, train_y, batch_size=20, shuffle=False
)  # noqa:E501
validation_generator = test_datagen.flow(
    val_X, val_y, batch_size=20, shuffle=False
)  # noqa:E501
test_generator = test_datagen.flow(
    test_X, test_y, batch_size=20, shuffle=False
)  # noqa:E501

In [ ]:
h.quick_plot_imggen(train_X[1], train_datagen)

# 3. Define the model

In [ ]:
model = models.Sequential()

# These are the deep layers that develop into feature extractors
model.add(
    layers.Conv2D(64, (3, 3), activation="relu", input_shape=(80, 80, 3))
)  # noqa:E501
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation="relu"))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), activation="relu"))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(512, (3, 3), activation="relu"))
model.add(layers.MaxPooling2D((2, 2)))

# And these are essentially a less complex classifier sat on top
# Note use of Dropout
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1024, activation="relu"))
model.add(layers.Dense(1, activation="sigmoid"))

# Not sure what specifically is computed at compilation, I guess this is where
# the backprop formulae etc are determined?
# DEPRECATION since book:  arg 'lr' replaced with 'learning_rate' for optimizer
# On a very small model 'steps_per_execution' can be raised,
# reducing python overhead
model.compile(
    loss="binary_crossentropy",
    optimizer=optimizers.RMSprop(learning_rate=1e-4),
    metrics=["acc"],
)

# View a summary - useful for improvising a build
model.summary()

# 4. Fit the model

In [ ]:
# Training loop#
# DEPRECATION since book:  method 'fit_generator', now just use 'fit'
# steps_per_epoch doesn't need to be specified if you've specified batch size
# in a generator based on a loaded dataset, but might be needed if you're
# streaming data from a directory.
# Likewise for validation_steps
history = model.fit(
    train_generator, epochs=50, validation_data=validation_generator
)  # noqa:E501

model.save("models/ship_spotting_v0.1.h5")

# 5. Review Performance

In [ ]:
# I'm going to use pandas and seaborn if I can for results plots
results = pd.DataFrame(history.history)

# Add 1 to the index values, so they go 1-30 rather than 0-29 (pure aesthetics)
results.index = results.index + 1

# Get rolling averages for a smoother look
for col in results.columns:
    results[col + "_rolling"] = results[col].rolling(3, center=True).mean()

print(results.tail(5))

In [ ]:
sns.lineplot(data=results[["acc_rolling", "val_acc_rolling"]])

In [ ]:
sns.lineplot(data=results[["loss_rolling", "val_loss_rolling"]])

In [ ]:
model.evaluate(test_generator)

# 6. Review (validation) predictions

In [ ]:
# Identify records in validation that are miss-identified
prob = model.predict(validation_generator).flatten()
pred = np.round(prob)

fails = []
for i in range(len(pred)):
    if pred[i] != val_y[i]:
        fails.append(i)

for example in fails:
    img = np.reshape(val_X[example], (80, 80, 3), order="F")
    plt.clf()
    plt.imshow((img))
    plt.text(
        2,
        5,
        f"True: {val_y[example]}, Pred: {int(pred[example])}",
        fontsize=15,  # noqa:E501
    )
    plt.savefig(os.path.join("outputs", "fails", f"validation_{example}.png"))

print("Fails output to file for examination")

In [ ]:
index = 1
h.quick_plot_img(val_X[index])

In [ ]:
layer_outputs = [layer.output for layer in model.layers[:8]]
activation_model = models.Model(inputs=model.input, outputs=layer_outputs)
activations = activation_model.predict(val_X)

In [ ]:
first_layer_activation = activations[0]
print(first_layer_activation.shape)
plt.matshow(first_layer_activation[index, :, :, 16], cmap="viridis")

In [ ]:
def visualise_activation(model, features):

    # Fetch raw layer output for each sample for every layer
    layer_outputs = [layer.output for layer in model.layers]
    activation_model = models.Model(inputs=model.input, outputs=layer_outputs)
    activations = activation_model.predict(features)

    layer_names = []
    for layer in model.layers:
        layer_names.append(layer.name)

    images_per_row = 16

    # Visualise each layer
    for layer_name, layer_activation in zip(layer_names, activations):
        if "2d" in layer_name:
            n_features = layer_activation.shape[-1]
            size = layer_activation.shape[1]
            n_cols = n_features // images_per_row
            display_grid = np.zeros((size * n_cols, images_per_row * size))

            for col in range(n_cols):
                for row in range(images_per_row):
                    channel_image = layer_activation[
                        0, :, :, col * images_per_row + row
                    ]

                    channel_image -= channel_image.mean()
                    channel_image /= channel_image.std()
                    channel_image *= 64
                    channel_image += 128
                    channel_image = np.clip(channel_image, 0, 255).astype(
                        "uint8"
                    )  # noqa:E501
                    display_grid[
                        col * size : (col + 1) * size,  # noqa:E203
                        row * size : (row + 1) * size,  # noqa:E203
                    ] = channel_image

            scale = 1.0 / size
            plt.figure(
                figsize=(
                    scale * display_grid.shape[1],
                    scale * display_grid.shape[0],
                )  # noqa:E501
            )
            plt.title(layer_name)
            plt.grid(False)
            plt.imshow(display_grid, aspect="auto", cmap="viridis")

    return None


visualise_activation(model, val_X)